In [1]:
import pandas as pd
df=pd.read_csv('nba_dataset/games_details_regular.csv')

In [2]:
team_stats=(df
            .groupby(['TEAM','GAME_ID'])[['FGM','FGA','FTA','OREB','TO','MIN']]
            .sum()
            .reset_index())

team_stats['team_poss']=(
    team_stats['FGA']+
    (0.44*team_stats['FTA'])-
    team_stats['OREB']+
    team_stats['TO']
).round(2)

team_stats=team_stats.rename(columns={'MIN':'team_minutes'})
team_stats=team_stats.rename(columns={'FGM':'team_FGM'})
team_stats['team_minutes']=team_stats['team_minutes'].round(2)

In [3]:
#team_stats와 game_details_season2022.merge 하기
df['MIN']=df['MIN'].round(2)
details = df.merge(
    team_stats[['GAME_ID','TEAM','team_poss','team_minutes','team_FGM']],
    on=['GAME_ID','TEAM'],
    how='left'
)

# 3. USG% 계산
df['MIN']=df['MIN'].round(2)
details = df.merge(
    team_stats[['GAME_ID','TEAM','team_poss','team_minutes','team_FGM']],
    on=['GAME_ID','TEAM'],
    how='left'
)

# 3. USG% 계산
details['USG%'] = 100 * (
    (details['FGA'] + (0.44*details['FTA']) + details['TO']) * details['team_minutes']
) / (details['MIN'] * details['team_poss'])

details['USG%'] = details['USG%'].round(2)

In [4]:
details['player_poss'] = (
    details['FGA'] + (0.44*details['FTA']) + details['TO']
)

details['PTS_per_poss'] = (details['PTS'] / details['player_poss']).round(2)


In [5]:
details['TS%']=details['PTS']/(2*(details['FGA']+0.44*details['FTA']))
details['TS%']=details['TS%'].round(2)

모든 득점 시도(2점, 3점, 자유투)를 반영한 실제 슈팅 효율

👉 “같은 시도 대비 얼마나 효율적으로 득점했는가?

In [6]:
details['TO%']=details['TO']/(details['FGA']+0.44*details['FTA']+details['TO'])
details['TO%']=details['TO%'].round(2)

In [7]:
# 4. AST% 계산
details['AST%'] = 100 * (
    details['AST'] 
) / (details['team_FGM']) 

details['AST%']=details['AST%'].round(2)
# 5. 선수별 평균 AST% 확인
ast_percent = details.groupby('PLAYER_ID')['AST%'].mean().round(2)
print(ast_percent.sort_values(ascending=False).head(10))

PLAYER_ID
1630169    25.73
201935     25.42
1629027    23.00
1629029    22.19
101108     21.65
203999     20.81
1629636    19.51
1630163    18.58
201950     18.45
1629630    18.00
Name: AST%, dtype: float64


In [8]:
details.drop('AST%', axis=1)

,DATE,GAME_ID,TEAM,TEAM_LVL,PLAYER_ID,START_POSITION,MIN,FGM,FGA,FG_PCT,...,PTS,PLUS_MINUS,team_poss,team_minutes,team_FGM,USG%,player_poss,PTS_per_poss,TS%,TO%
0,2022-10-18,22200001,Boston,상위팀,1630573,0,3.32,0,0,0.000,...,0,3,98.32,239.98,46,0.00,0.00,NaN,NaN,NaN
1,2022-10-18,22200001,Boston,상위팀,1628369,F,38.62,13,20,0.650,...,35,10,98.32,239.98,46,170.39,26.96,1.30,0.73,0.11
2,2022-10-18,22200001,Boston,상위팀,201933,0,8.28,0,2,0.000,...,1,-5,98.32,239.98,46,84.90,2.88,0.35,0.17,0.00
3,2022-10-18,22200001,Boston,상위팀,1627763,0,23.87,7,11,0.636,...,16,9,98.32,239.98,46,131.70,12.88,1.24,0.67,0.08
4,2022-10-18,22200001,Boston,상위팀,1629684,0,23.95,5,5,1.000,...,15,-3,98.32,239.98,46,64.41,6.32,2.37,1.19,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10178,2022-12-22,22200478,Washington,하위팀,201959,0,25.03,3,3,1.000,...,6,-3,96.48,240.00,46,29.81,3.00,2.00,1.00,0.00
10179,2022-12-22,22200478,Washington,하위팀,1628420,G,30.27,2,5,0.400,...,5,3,96.48,240.00,46,61.14,7.44,0.67,0.46,0.27
10180,2022-12-22,22200478,Washington,하위팀,203078,G,31.08,13,20,0.650,...,30,4,96.48,240.00,46,190.17,23.76,1.26,0.69,0.08
10181,2022-12-22,22200478,Utah,중위팀,1631117,C,32.38,5,7,0.714,...,12,16,98.24,240.00,42,67.00,8.88,1.35,0.76,0.11


In [9]:
import pandas as pd
import numpy as np

df1 = details.copy()


# BBR 공식: AST% = 100 * AST / [ ((MIN / (team_minutes/5)) * team_FGM) - FGM ]
den = ((df1['MIN'] / (df1['team_minutes'] / 5.0)) * df1['team_FGM']) - df1['FGM']
df1['AST_pct_bbr'] = (100.0 * df1['AST'] / den.replace(0, np.nan)).clip(0, 100)

In [10]:
df1['AST_pct_bbr']=df1['AST_pct_bbr'].round(2)

In [11]:
df1

,DATE,GAME_ID,TEAM,TEAM_LVL,PLAYER_ID,START_POSITION,MIN,FGM,FGA,FG_PCT,...,team_poss,team_minutes,team_FGM,USG%,player_poss,PTS_per_poss,TS%,TO%,AST%,AST_pct_bbr
0,2022-10-18,22200001,Boston,상위팀,1630573,0,3.32,0,0,0.000,...,98.32,239.98,46,0.00,0.00,NaN,NaN,NaN,0.00,0.00
1,2022-10-18,22200001,Boston,상위팀,1628369,F,38.62,13,20,0.650,...,98.32,239.98,46,170.39,26.96,1.30,0.73,0.11,8.70,16.66
2,2022-10-18,22200001,Boston,상위팀,201933,0,8.28,0,2,0.000,...,98.32,239.98,46,84.90,2.88,0.35,0.17,0.00,2.17,12.60
3,2022-10-18,22200001,Boston,상위팀,1627763,0,23.87,7,11,0.636,...,98.32,239.98,46,131.70,12.88,1.24,0.67,0.08,8.70,25.19
4,2022-10-18,22200001,Boston,상위팀,1629684,0,23.95,5,5,1.000,...,98.32,239.98,46,64.41,6.32,2.37,1.19,0.00,2.17,5.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10178,2022-12-22,22200478,Washington,하위팀,201959,0,25.03,3,3,1.000,...,96.48,240.00,46,29.81,3.00,2.00,1.00,0.00,2.17,4.76
10179,2022-12-22,22200478,Washington,하위팀,1628420,G,30.27,2,5,0.400,...,96.48,240.00,46,61.14,7.44,0.67,0.46,0.27,8.70,14.81
10180,2022-12-22,22200478,Washington,하위팀,203078,G,31.08,13,20,0.650,...,96.48,240.00,46,190.17,23.76,1.26,0.69,0.08,10.87,29.79
10181,2022-12-22,22200478,Utah,중위팀,1631117,C,32.38,5,7,0.714,...,98.24,240.00,42,67.00,8.88,1.35,0.76,0.11,2.38,4.29


In [12]:
# 1. 상위팀 & 선발 포지션만 필터링
pos = ['F','G','C']
top_details = df1[
    (df1['TEAM_LVL'] == '상위팀') &
    (df1['START_POSITION'].isin(pos))
].copy()

# 2. USG% × MIN (공격권 총량) 계산
top_details['USG_x_MIN'] = top_details['USG%'] * top_details['MIN']

# 3. 팀-포지션별 합계
team_pos = (
    top_details.groupby(['TEAM','START_POSITION'])['USG_x_MIN']
    .sum()
    .reset_index()
)

# 4. 팀별 총합
team_total = (
    team_pos.groupby('TEAM')['USG_x_MIN']
    .sum()
    .rename('TEAM_total')
    .reset_index()
)

# 5. 팀별 포지션 비중 계산
team_pos = team_pos.merge(team_total, on='TEAM', how='left')
team_pos['POS_share'] = team_pos['USG_x_MIN'] / team_pos['TEAM_total']

# 6. 포지션별 평균 (상위팀 전체 평균)
pos_avg = (
    team_pos.groupby('START_POSITION')['POS_share']
    .mean()
    .reset_index()
    .sort_values('POS_share', ascending=False)
)

print(pos_avg)

  START_POSITION  POS_share
2              G   0.415335
1              F   0.411104
0              C   0.173561


In [13]:
pos = ['F','G','C']

# 1. DET + 선발 포지션만 뽑기
det_details = df1[
    (df1['TEAM'] == 'Detroit') & 
    (df1['START_POSITION'].isin(pos))
].copy()

# 2. USG% × MIN 계산
det_details['USG_x_MIN'] = det_details['USG%'] * det_details['MIN']

# 3. 포지션별 합계
det_pos = (
    det_details.groupby('START_POSITION')['USG_x_MIN']
    .sum()
    .reset_index()
)

# 4. 전체 합 대비 비중
total = det_pos['USG_x_MIN'].sum()
det_pos['POS_share'] = det_pos['USG_x_MIN'] / total

print(det_pos)


  START_POSITION    USG_x_MIN  POS_share
0              C   84526.9266   0.143869
1              F  243090.7941   0.413753
2              G  259909.1393   0.442378


🏀 Basketball Reference의 AST% 공식

AST% = 100 × AST ÷ (FGM – FG_self)
여기서

AST = 선수가 기록한 어시스트 수

FGM = 팀이 선수가 뛴 동안 넣은 필드골 수

FG_self = 선수가 직접 넣은 필드골

즉,
👉 “선수가 뛴 동안 팀이 성공한 슛 중, 내가 직접 넣은 걸 빼고 나머지 중 몇 개를 내가 어시스트했는가?”
분모 처리 차이

bbr: 팀 FGM – 내 FGM

내가 넣은 슛은 내가 어시스트할 수 없으니까 빼야 함

출전 시간 반영

bbr: 선수가 뛴 시간 동안의 팀 필드골만 반영

단순 계산: 팀 전체 시즌 필드골로 나누면 왜곡됨

데이터 출처 차이

데이터셋에 “내가 뛴 시간 동안 팀 전체 필드골” 정보가 없으면 정확히 bbr 공식을 못 씀

In [14]:
top_details.drop('AST%',axis=1)

,DATE,GAME_ID,TEAM,TEAM_LVL,PLAYER_ID,START_POSITION,MIN,FGM,FGA,FG_PCT,...,team_poss,team_minutes,team_FGM,USG%,player_poss,PTS_per_poss,TS%,TO%,AST_pct_bbr,USG_x_MIN
1,2022-10-18,22200001,Boston,상위팀,1628369,F,38.62,13,20,0.650,...,98.32,239.98,46,170.39,26.96,1.30,0.73,0.11,16.66,6580.4618
6,2022-10-18,22200001,Boston,상위팀,1628401,G,24.03,1,3,0.333,...,98.32,239.98,46,45.10,4.44,0.45,0.29,0.23,18.16,1083.7530
7,2022-10-18,22200001,Boston,상위팀,201143,C,23.10,2,7,0.286,...,98.32,239.98,46,73.96,7.00,0.86,0.43,0.00,4.97,1708.4760
8,2022-10-18,22200001,Boston,상위팀,1627759,F,38.57,14,24,0.583,...,98.32,239.98,46,185.54,29.32,1.19,0.69,0.14,8.71,7156.2778
9,2022-10-18,22200001,Philadelphia,상위팀,1630178,G,38.20,8,16,0.500,...,102.32,240.00,40,112.49,18.32,1.15,0.61,0.05,8.39,4297.1180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10133,2022-12-22,22200477,New Orleans,상위팀,1630530,F,27.78,5,8,0.625,...,103.88,240.00,44,84.83,10.20,1.47,0.74,0.00,4.89,2356.5774
10138,2022-12-22,22200477,New Orleans,상위팀,203468,G,37.18,14,27,0.519,...,103.88,240.00,44,212.52,34.20,1.17,0.68,0.15,44.82,7901.4936
10139,2022-12-22,22200477,New Orleans,상위팀,1630529,G,34.43,4,8,0.500,...,103.88,240.00,44,73.01,10.88,1.10,0.68,0.18,7.26,2513.7343
10140,2022-12-22,22200477,New Orleans,상위팀,202685,C,25.83,6,14,0.429,...,103.88,240.00,44,158.85,17.76,0.90,0.51,0.11,5.66,4103.0955


In [15]:
df1.to_csv('df1.csv')

In [16]:
det = df1[df1["TEAM"] == "Detroit"].copy()

det_player = det.groupby("PLAYER_ID", as_index=False).agg(
    PTS_sum=("PTS","sum"),
    poss_sum=("player_poss","sum")
)
det_player["PPP"] = det_player["PTS_sum"] / det_player["poss_sum"]

print(det_player.sort_values("PPP", ascending=False).head(5))  # 효율 높은 상위 5명


    PLAYER_ID  PTS_sum  poss_sum       PPP
11    1630296        4      1.88  2.127660
2      202711      684    605.12  1.130354
0      202692      279    252.16  1.106440
5     1628963      232    211.24  1.098277
7     1628995      161    147.12  1.094345


In [17]:
players=pd.read_csv('nba_dataset/games_details.csv')

/var/folders/vf/9p8syxcx1g77s8jzp8b8b8fh0000gn/T/ipykernel_79381/2354249120.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  players=pd.read_csv('nba_dataset/games_details.csv')


In [18]:
player = players[['PLAYER_NAME','PLAYER_ID']]


In [19]:
# 필요한 컬럼만 남긴 매핑 테이블
player = players[['PLAYER_NAME','PLAYER_ID']]

# df1과 합치기 (PLAYER_ID 기준)
df1_named = df1.merge(player, on='PLAYER_ID', how='left')

print(df1_named.isna().sum())


DATE                   0
GAME_ID                0
TEAM                   0
TEAM_LVL               0
PLAYER_ID              0
START_POSITION         0
MIN                    0
FGM                    0
FGA                    0
FG_PCT                 0
FG3M                   0
FG3A                   0
FG3_PCT                0
FTM                    0
FTA                    0
FT_PCT                 0
OREB                   0
DREB                   0
REB                    0
AST                    0
STL                    0
BLK                    0
TO                     0
PF                     0
PTS                    0
PLUS_MINUS             0
team_poss              0
team_minutes           0
team_FGM               0
USG%                   0
player_poss            0
PTS_per_poss       91376
TS%               121890
TO%                91376
AST%                   0
AST_pct_bbr            0
PLAYER_NAME            0
dtype: int64


In [20]:
import numpy as np
import pandas as pd

# -----------------------------
# 1) USG% 계산 (possession 기반, 올바른 스케일)
#    USG% = 100 * (FGA + 0.44*FTA + TO) * 48 / (MIN * team_poss)
#    ※ team_poss 를 분모로 쓸 때는 48(= 팀 평균 개인출전시간)로 스케일링해야 함
# -----------------------------
num_cols = ['FGA','FTA','TO','MIN','team_poss']
for c in num_cols:
    df1_named[c] = pd.to_numeric(df1_named[c], errors='coerce')

num = (df1_named['FGA'] + 0.44*df1_named['FTA'] + df1_named['TO']) * 48
den = (df1_named['MIN'] * df1_named['team_poss'])

df1_named['USG%'] = np.where(den > 0, 100 * num / den, np.nan).round(2)

# -----------------------------
# 2) 디트로이트 "선발(C/F/G)"만 필터
#    (팀 컬럼명이 다른 경우: TEAM_ABBREVIATION=='DET' 등으로 바꿔주세요)
# -----------------------------
valid_pos = ['C','F','G']
det_details = df1_named[
    (df1_named['TEAM'] == 'Detroit') &
    (df1_named['START_POSITION'].isin(valid_pos))
].copy()

# -----------------------------
# 3) 시간가중 USG (USG% × MIN) 집계
# -----------------------------
det_details['USG_x_MIN'] = det_details['USG%'] * det_details['MIN']

player_pos_usg = (
    det_details
      .groupby(['PLAYER_ID','PLAYER_NAME','START_POSITION'], observed=True)
      .agg(
          USG_minutes_sum = ('USG_x_MIN','sum'),    # Σ(USG% × MIN)
          MIN_sum         = ('MIN','sum'),          # Σ MIN
          games           = ('GAME_ID','nunique')   # 경기 수
      )
      .assign(
          USG_wavg = lambda d: np.where(d['MIN_sum']>0,
                                        d['USG_minutes_sum']/d['MIN_sum'],
                                        np.nan)
      )
      .reset_index()
      .sort_values(['START_POSITION','USG_wavg'], ascending=[True, False])
)

# 보기 좋게 반올림
player_pos_usg['USG_wavg']       = player_pos_usg['USG_wavg'].round(2)
player_pos_usg['USG_minutes_sum'] = player_pos_usg['USG_minutes_sum'].round(2)

print(player_pos_usg.head(10))


    PLAYER_ID        PLAYER_NAME START_POSITION  USG_minutes_sum    MIN_sum  \
3     1628963  Marvin Bagley III              C       1544577.83   71409.66   
6     1630191     Isaiah Stewart              C       1346498.27   72245.76   
12    1631105        Jalen Duren              C         93124.32    7187.60   
1      202711   Bojan Bogdanovic              F      20740067.42  742522.95   
5     1630180         Saddiq Bey              F       2334739.67  104691.71   
7     1630191     Isaiah Stewart              F       1219341.17   65905.12   
8     1630587      Isaiah Livers              F         70352.36    5995.05   
10    1630595    Cade Cunningham              G       1214392.34   36816.56   
11    1631093         Jaden Ivey              G        841892.94   30650.20   
4     1630165      Killian Hayes              G       1899933.19   86633.69   

    games  USG_wavg  
3      13     21.63  
6      14     18.64  
12      7     12.96  
1      33     27.93  
5      17     22.30 

In [21]:

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# 색상 팔레트 (큰 비중부터 차례대로)
custom_colors = ["#d73027", "#4575b4", "#f7f7f7", "#000000"]

positions = ['C','F','G']
subfig = make_subplots(
    rows=1, cols=len(positions),
    specs=[[{"type":"domain"}]*len(positions)],
    subplot_titles=positions
)

for i, pos in enumerate(positions, start=1):
    subdf = player_pos_usg[player_pos_usg['START_POSITION'] == pos]

    # 비중(USG_minutes_sum) 큰 순서대로 정렬
    subdf = subdf.sort_values("USG_minutes_sum", ascending=False).reset_index(drop=True)

    # 색상: 큰 비중부터 빨강-파랑-흰-검정 순서로 할당
    colors = custom_colors * (len(subdf) // len(custom_colors) + 1)

    subfig.add_trace(
        go.Pie(
            labels=subdf['PLAYER_NAME'],
            values=subdf['USG_minutes_sum'],
            name=pos,
            textinfo="percent",
            textposition="inside",
            showlegend=True,
            marker=dict(colors=colors[:len(subdf)])
        ),
        row=1, col=i
    )
subfig.update_layout(
    title_text="DET 포지션별: 선수별 공격권 비중 (USG%×MIN)",
    legend_title="선수"
)

subfig.show()


In [22]:
import plotly.express as px

# 디트로이트 팀만
det = df1_named[df1_named['TEAM'] == 'Detroit'].copy()

# 선수별 평균 PPP 구하기
det_player = det.groupby('PLAYER_NAME')['PTS_per_poss'].mean().round(2).reset_index()
print(det_player.info())
det_player['PLAYER_NAME'] = det_player['PLAYER_NAME'].astype(str)
det1=det_player.sort_values('PTS_per_poss', ascending=False)
# 막대그래프 그리기
fig = px.bar(det1,
             x='PLAYER_NAME',
             y='PTS_per_poss',
             title="Detroit 선수별 평균 PTS per Poss",
            )
fig.show()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER_NAME   17 non-null     object 
 1   PTS_per_poss  16 non-null     float64
dtypes: float64(1), object(1)
memory usage: 400.0+ bytes
None


In [23]:
import pandas as pd
import numpy as np

# 1) 상위팀만 필터
top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

# 3) 팀별 총합 (득점/포제션)
top_team = top.groupby('TEAM', as_index=False).agg(
    PTS_sum=('PTS','sum'),
    poss_sum=('player_poss','sum')
)
top_team['PPP'] = np.where(top_team['poss_sum']>0,
                           top_team['PTS_sum']/top_team['poss_sum'], 0)

print("=== 상위팀별 PPP ===")
print(top_team[['TEAM','PPP']].sort_values('PPP', ascending=False))

# 4) 상위팀 평균 PPP
top_avg_ppp = top_team['PPP'].mean()
print("\n=== 상위팀 평균 PPP ===")
print(f"{top_avg_ppp:.3f}")


=== 상위팀별 PPP ===
           TEAM       PPP
1      Brooklyn  1.099763
3        Denver  1.062903
0        Boston  1.062050
2     Cleveland  1.033071
7  Philadelphia  1.026947
6   New Orleans  1.001894
5     Milwaukee  0.997720
4       Memphis  0.976648

=== 상위팀 평균 PPP ===
1.033


In [24]:
import numpy as np

# 1) 디트로이트만 필터
pos = ['C','F','G']
det_starters = df1[
    (df1['TEAM'].str.contains("Detroit", case=False, na=False)) &
    (df1['START_POSITION'].isin(pos))
].copy()


# 2) 숫자형 변환
for c in ['PTS','player_poss']:
    det[c] = pd.to_numeric(det[c], errors='coerce').fillna(0)

# 3) 팀 전체 PPP = 총득점 / 총포제션
det_total_pts = det['PTS'].sum()
det_total_poss = det['player_poss'].sum()

det_ppp = det_total_pts / det_total_poss if det_total_poss > 0 else 0

print("=== 디트로이트 팀 PPP ===")
print(f"총득점: {det_total_pts}, 총포제션: {det_total_poss}")
print(f"팀 PPP: {det_ppp:.3f}")


=== 디트로이트 팀 PPP ===
총득점: 1259524, 총포제션: 1215905.52
팀 PPP: 1.036


In [25]:
print(f"디트로이트 PPP: {det_ppp:.3f}")
print(f"상위팀 평균 PPP: {top_avg_ppp:.3f}")


디트로이트 PPP: 1.036
상위팀 평균 PPP: 1.033


In [26]:
# 경기+팀별 포제션 합
game_team_poss = df1.groupby(['GAME_ID','TEAM'], as_index=False)['player_poss'].sum()

# 팀별 경기 포제션 평균
avg_poss = game_team_poss['player_poss'].mean()
print("팀당 경기당 평균 포제션:", f"{avg_poss:.3f}")



팀당 경기당 평균 포제션: 112.503


In [27]:

data={
    '디트로이트 ppp':0.978,
    '상위팀 평균 ppp':1.028
}
ppp=pd.Series(data).reset_index()
ppp.columns = ['팀', 'PPP']
fig=px.bar(
    ppp,
    x='팀',
    y='PPP' ,
    title="상위팀과 디트로이트 팀의 PTS per Possession 비교"
)

fig.show()

PPP 격차 0.05 → 112포제션 기준, 경기당 약 5~6점 차이 발생”

“작은 차이가 누적되면 시즌 전체 수백 점 차이, 하위권 성적의 원인”

In [28]:
player = players[['PLAYER_ID','PLAYER_NAME']].drop_duplicates('PLAYER_ID')
df1_named = df1.merge(player, on='PLAYER_ID', how='left')
df1_named

,DATE,GAME_ID,TEAM,TEAM_LVL,PLAYER_ID,START_POSITION,MIN,FGM,FGA,FG_PCT,...,team_minutes,team_FGM,USG%,player_poss,PTS_per_poss,TS%,TO%,AST%,AST_pct_bbr,PLAYER_NAME
0,2022-10-18,22200001,Boston,상위팀,1630573,0,3.32,0,0,0.000,...,239.98,46,0.00,0.00,NaN,NaN,NaN,0.00,0.00,Sam Hauser
1,2022-10-18,22200001,Boston,상위팀,1628369,F,38.62,13,20,0.650,...,239.98,46,170.39,26.96,1.30,0.73,0.11,8.70,16.66,Jayson Tatum
2,2022-10-18,22200001,Boston,상위팀,201933,0,8.28,0,2,0.000,...,239.98,46,84.90,2.88,0.35,0.17,0.00,2.17,12.60,Blake Griffin
3,2022-10-18,22200001,Boston,상위팀,1627763,0,23.87,7,11,0.636,...,239.98,46,131.70,12.88,1.24,0.67,0.08,8.70,25.19,Malcolm Brogdon
4,2022-10-18,22200001,Boston,상위팀,1629684,0,23.95,5,5,1.000,...,239.98,46,64.41,6.32,2.37,1.19,0.00,2.17,5.57,Grant Williams
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10178,2022-12-22,22200478,Washington,하위팀,201959,0,25.03,3,3,1.000,...,240.00,46,29.81,3.00,2.00,1.00,0.00,2.17,4.76,Taj Gibson
10179,2022-12-22,22200478,Washington,하위팀,1628420,G,30.27,2,5,0.400,...,240.00,46,61.14,7.44,0.67,0.46,0.27,8.70,14.81,Monte Morris
10180,2022-12-22,22200478,Washington,하위팀,203078,G,31.08,13,20,0.650,...,240.00,46,190.17,23.76,1.26,0.69,0.08,10.87,29.79,Bradley Beal
10181,2022-12-22,22200478,Utah,중위팀,1631117,C,32.38,5,7,0.714,...,240.00,42,67.00,8.88,1.35,0.76,0.11,2.38,4.29,Walker Kessler


In [29]:
# 1) 상위팀 + 선발(C/F/G)만 필터
pos = ['C','F','G']
top_starters = df1_named[(df1_named['TEAM_LVL'] == '상위팀') &
                   (df1_named['START_POSITION'].isin(pos))].copy()

# 2) 선수별 + 포지션별 TS% 평균 계산
player_pos_ts = top_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['TS%'].mean()

In [31]:
# 1) 상위팀 + 선발(C/F/G)만 필터
pos = ['C','F','G']
top_starters = df1_named[(df1_named['TEAM_LVL'] == '상위팀') &
                   (df1_named['START_POSITION'].isin(pos))].copy()

# 2) 상위팀 선발 전체의 TS% 평균
top_starters_ts = top_starters['TS%'].mean()

print("상위팀 선발 평균 TS%:", round(top_starters_ts, 3))


상위팀 선발 평균 TS%: 0.591


In [32]:
# 1) 선발 포지션 정의
pos = ['C','F','G']

# 2) 디트로이트 팀 + 선발만 필터
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 3) TS% 평균 계산
det_starters_ts = det_starters['TS%'].mean()

print("디트로이트 선발 평균 TS%:", round(det_starters_ts, 3))

디트로이트 선발 평균 TS%: 0.561


In [33]:
data={
    '디트로이트 선발 평균 TS%': 0.561,
    '상위팀 선발 평균 TS%': 0.591
}
ts=pd.Series(data).reset_index()
ts.columns = ['팀', 'TS%']
fig=px.bar(
    ts,
    x='팀',
    y='TS%',
    title="상위팀 vs 디트로이트 팀 TS% 비교"
)
fig.show()

**Tempo(포제션 수)**는 리그 평균 수준 → 문제는 “속도”가 아님

**효율(PPP)**에서 상위팀보다 뒤쳐짐 → 매 경기 점수 손실

결론: 디트로이트의 문제는 경기 템포가 아니라 공격권을 어떻게 활용했는가

In [34]:
import plotly.express as px

# 디트로이트 선발만 필터
pos = ['C','F','G']
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 선수별 TS% 평균 (포지션 포함)
det_player_ts = det_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['TS%'].mean()

# 산점도 (TS% 크기로 반영)
fig = px.scatter(
    det_player_ts,
    x='PLAYER_NAME',
    y='TS%',
    color='START_POSITION',   # 포지션별 색 구분
    size='TS%',
    title="디트로이트 선발 선수별 TS% 분포 (포지션별 구분)",
    labels={'START_POSITION': 'Position'},   # 범례 제목 바꾸기
    category_orders={'START_POSITION': ['C','F','G']}  # 범례 순서 고정
)

# 범례 항목별 레이블 직접 바꾸기
fig.for_each_trace(lambda t: t.update(
    name = t.name.replace("C","Center").replace("F","Forward").replace("G","Guard")
))
fig.show()



1. 분포 자체의 차이

상위팀 선발: TS%가 0.55~0.65 이상에 고르게 몰려 있음
디트로이트 선발: 0.4~0.55 구간에 많이 분포 → 효율이 낮은 선수 다수 존재
결론: 디트로이트는 팀 전체 효율을 깎아먹는 저효율 선수가 눈에 띄게 많음

2. 포지션별 차이

가드(G)

상위팀: 대부분 0.55~0.6 이상 → 안정적으로 효율을 유지
디트로이트: 0.1~0.5 사이로 넓게 퍼져 있음 → 불안정, 특히 낮은 효율 선수 존재

포워드(F)

상위팀: 대부분 리그 평균 수준(0.55~0.6 이상)
디트로이트: 일부는 준수하지만 고른 분포가 아님

센터(C)

상위팀: 안정적으로 0.6 근처 이상
디트로이트: 일부 괜찮지만 표본 적고 편차 큼

3. 전략적 시사점

상위팀은 누가 들어와도 효율 유지 → 고른 분포 = 전술적 안정성
디트로이트는 특정 선수(특히 가드)에 의존 → 효율 편차 큼 = 전술적 불안정성
효율이 낮은 선수가 주전으로 많은 시간을 뛰면 → 전체 팀 효율 하락으로 이어짐

“Isaiah Livers는 팀 내 공격 점유율(USG%)은 낮지만, 주어진 기회에서 높은 TS%를 기록했습니다. 이는 공격의 메인 옵션은 아니지만, 효율적인 보조 득점원으로서 가치를 보여줍니다.”

Isaiah Livers 케이스 정리

USG% 낮음

팀 내에서 공격 기회는 많이 받지 못함 → 즉, 전술적으로 중심은 아님
하지만 이는 “아직 비중이 적다”는 의미일 뿐, 발전 가능성을 배제하진 않음

TS% 높음

주어진 기회를 효율적으로 마무리
보통 이런 선수는 오픈 찬스를 잘 살리고, 슈팅 셀렉션이 안정적임

해석

지금은 보조 옵션(롤플레이어) 성격이 강하지만,
만약 USG%를 조금 더 올려도 TS%가 유지된다면, 더 큰 역할을 맡을 수 있음
즉, “효율이 검증된 롤플레이어 → 성장 시 2옵션급 선수”로 발전할 가능성

✅ 발표/분석용 메시지

“Isaiah Livers는 현재는 낮은 점유율(USG%)에도 불구하고 높은 TS%를 기록하며 효율적인 득점원으로서 잠재력을 보였습니다. 만약 더 많은 공격 기회를 부여해도 효율을 유지할 수 있다면, 디트로이트가 키워볼 만한 성장 자원으로 평가할 수 있습니다.”


In [36]:


# 디트로이트 팀만 필터
det = df1_named[df1_named['TEAM'].str.contains("Detroit", case=False, na=False)].copy()

# 상위팀만 필터
top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

# 평균 TO% 계산
det_to_mean = det['TO%'].mean()
top_to_mean = top['TO%'].mean()

print("디트로이트 평균 TO%:", round(det_to_mean, 3))
print("상위팀 평균 TO%:", round(top_to_mean, 3))

data = {
    '팀': ['디트로이트', '상위팀'],
    'TO% 평균': [det_to_mean, top_to_mean]
}
df_plot = pd.DataFrame(data)

fig = px.bar(df_plot,
             x='팀',
             y='TO% 평균',
             title="디트로이트 vs 상위팀 TO% 평균 비교",
             text='TO% 평균')

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_yaxes(range=[0,0.15])
fig.show()

디트로이트 평균 TO%: 0.127
상위팀 평균 TO%: 0.133


거의 차이가 없으니, “팀 전체 평균만 보면” 디트로이트가 턴오버 때문에 약하다고 단정할 수 없음.

In [37]:

# 선발 포지션
pos = ['C','F','G']

# 디트로이트 선발 선수만
det_starters = df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]

# 선수별 평균 TO%
det_starters_to = det_starters.groupby('PLAYER_NAME', as_index=False)['TO%'].mean()

# 시각화
fig = px.bar(det_starters_to,
             x='PLAYER_NAME',
             y='TO%',
             title="디트로이트 선발 선수별 평균 TO%",
             text='TO%')
fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(xaxis_tickangle=45)
fig.update_yaxes(range=[0,0.25])
fig.show()



평균적으로는 리그 상위팀과 큰 차이가 없음 (DET 12.7%, TOP 13.3%).

하지만 선수별로 보면 Jaden Ivey (0.199 ≈ 20%) 등 특정 주전의 TO%가 지나치게 높음 → 공격권 낭비가 집중적으로 발생.
반면 보조 자원(Isaiah Livers 등)은 낮은 점유율에도 높은 효율을 보임 → 역할/공격권 재분배 필요성.

즉, “에이스 의존 + 낮은 효율 + 특정 선수 턴오버 집중” 이 세 가지가 팀이 하위권에 머무르는 주요 원인

In [38]:

# 디트로이트 팀만 필터
det = df1_named[df1_named['TEAM']=="Detroit"].copy()

# 상위팀만 필터
top = df1_named[df1_named['TEAM_LVL'] == '상위팀'].copy()

# 평균 AST% 계산
det_ast_mean = det['AST%'].mean()
top_ast_mean = top['AST%'].mean()

print("디트로이트 평균 AST%:", round(det_ast_mean, 3))
print("상위팀 평균 AST%:", round(top_ast_mean, 3))

# 시각화
data = {
    '팀': ['디트로이트', '상위팀'],
    'AST% 평균': [det_ast_mean, top_ast_mean]
}
df_plot = pd.DataFrame(data)

fig = px.bar(df_plot,
             x='팀',
             y='AST% 평균',
             title="디트로이트 vs 상위팀 AST% 평균 비교",
             text='AST% 평균')

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_yaxes(range=[0,7])
fig.show()


디트로이트 평균 AST%: 5.201
상위팀 평균 AST%: 5.728


In [39]:
import plotly.express as px

# 디트로이트 선수만 추출
pos = ['C','F','G']

# 디트로이트 선발 선수만
det= df1_named[
    (df1_named['TEAM']=="Detroit") &
    (df1_named['START_POSITION'].isin(pos))
]
# 선수별 평균 AST%
det_ast = det.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

# 산점도
fig = px.scatter(det_ast,
                 x='PLAYER_NAME',
                 y='AST%',
                 color='START_POSITION',  # 포지션별 색 구분
                 title="디트로이트 선수별 AST% 분포 (포지션별)",
                 size='AST%',             # 점 크기로 기여도 표현
                 hover_data=['START_POSITION'])

fig.update_layout(
    xaxis_tickangle=45,
    yaxis=dict(title="AST%"),
    width=950,
    height=600
)

fig.show()


In [40]:
import plotly.express as px

# 선발 포지션 정의
pos = ['C','F','G']

# 상위팀 + 선발만 필터
top_starters = df1_named[
    (df1_named['TEAM_LVL'] == '상위팀') &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 선수별 평균 TS% 계산
top_player_ts = top_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['TS%'].mean()


# 산점도
fig = px.scatter(
    top_player_ts,
    x='PLAYER_NAME',
    y='TS%',
    color='START_POSITION',   # 포지션별 색 구분
    size='TS%',               # TS% 크기 반영 (시각적 강조)
    hover_data=['START_POSITION'],
    title="상위팀 선발 선수별 TS% 분포"
)

fig.for_each_trace(lambda t: t.update(name = t.name.replace("C","Center").replace("F","Forward").replace("G","Guard")))
fig.update_traces(marker=dict(line=dict(width=1, color='black')))
fig.update_layout(xaxis_tickangle=45, yaxis_title="TS%", width=1000, height=600)
fig.show()


In [41]:
import plotly.express as px

# 선발 포지션 정의
pos = ['C','F','G']

# 상위팀 + 선발만 필터
top_starters = df1_named[
    (df1_named['TEAM_LVL'] == '상위팀') &
    (df1_named['START_POSITION'].isin(pos))
].copy()

# 선수별 평균 AST% 계산
top_player_ast = top_starters.groupby(['PLAYER_NAME','START_POSITION'], as_index=False)['AST%'].mean()

# 산점도
fig = px.scatter(
    top_player_ast,
    x='PLAYER_NAME',
    y='AST%',
    color='START_POSITION',   # 포지션별 색 구분
    size='AST%',              # AST% 크기 반영
    hover_data=['START_POSITION'],
    title="상위팀 선발 선수별 AST% 분포"
)

fig.update_traces(marker=dict(line=dict(width=1, color='black')))
fig.update_layout(xaxis_tickangle=45, yaxis_title="AST%", width=1000, height=600)
fig.show()
